In [1]:
# Import packages
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import h5py
import requests
import geopandas as gpd
import earthpy.plot as ep

project_path = os.path.abspath(os.path.join('..'))
#print(project_path)
if project_path not in sys.path:
    sys.path.append(project_path)

import modules.reflectance as refl

In [2]:
# Get NEON reflectance data from NEON API
# Endpoints are data product, site, date, and release year
neon_base = "https://data.neonscience.org/api/v0/data"
data_product = "/DP3.30006.001"
site = "/GRSM"
post_fire_release = "/2017-10?package=basic&release=RELEASE-2022"
post_fire_url = neon_base + data_product + site + post_fire_release
post_fire_data = requests.get(post_fire_url)

# Similarly for the pre-fire data
pre_fire_release = "/2016-06?package=basic&release=RELEASE-2022"
pre_fire_url = neon_base + data_product + site + pre_fire_release
pre_fire_data = requests.get(pre_fire_url)

In [3]:
# Define data directory and data paths
data_dir = os.path.join(project_path, 'data')
post_data_path = os.path.join(
    data_dir, 'NEON_GRSM_274000_3947000_201710_reflectance.h5')
pre_data_path = os.path.join(
    data_dir, 'NEON_GRSM_274000_3947000_201606_reflectance.h5')

# # Check if data directory exists
# if not os.path.exists(data_dir):
#     os.makedirs(data_dir)

In [4]:
# Check if data directory exists
try:
    os.makedirs(data_dir)
    print('The data directory has been created!')
except FileExistsError:
    print('The data directory already exists!')
except NameError:
    print('Please check the name of the directory!')

The data directory already exists!


In [5]:
tile_url = (
    'https://storage.googleapis.com/neon-aop-products/2017/FullSite/D07/'
    '2017_GRSM_3/L3/Spectrometer/Reflectance/'
    'NEON_D07_GRSM_DP3_272000_3952000_reflectance.h5')
tile_path = os.path.join(
    data_dir, 'NEON_D07_GRSM_DP3_272000_3952000_201710_reflectance.h5')
tile_data = refl.download_file(tile_path, tile_url)

In [6]:
tile_refl, tile_metadata = refl.aop_h5refl2array(tile_path)

In [7]:
tile_b58 = tile_refl[:,:,57].astype(float)

In [ ]:
tile_plot = plt.imshow(tile_b58,extent=tile_metadata['extent'],cmap='Greys') 

In [ ]:
# Get NEON reflectance data
post_url = (
        'https://storage.googleapis.com/neon-aop-products/2017/FullSite/D07/'
        '2017_GRSM_3/L3/Spectrometer/Reflectance/'
        'NEON_D07_GRSM_DP3_274000_3947000_reflectance.h5')
pre_url = (
        'https://storage.googleapis.com/neon-aop-products/2016/FullSite/D07/'
        '2016_GRSM_2/L3/Spectrometer/Reflectance/'
        'NEON_D07_GRSM_DP3_274000_3947000_reflectance.h5')

post_fire_data = refl.download_file(post_data_path, post_url)
pre_fire_data = refl.download_file(pre_data_path, pre_url)
# if not os.path.exists(post_data_path):
#     # Download and save post-fire data if needed
#     post_url = (
#         'https://storage.googleapis.com/neon-aop-products/2017/FullSite/D07/'
#         '2017_GRSM_3/L3/Spectrometer/Reflectance/'
#         'NEON_D07_GRSM_DP3_274000_3947000_reflectance.h5')
#     post_r = requests.get(post_url)
#     with open(post_data_path, 'wb') as post_data_file:
#         post_data_file.write(post_r.content)
#         post_data = post_r.content
# else:
#     # Read post-fire data
#     with open(post_data_path, 'rb') as post_data_file:
#         post_data = post_data_file.read()

In [ ]:
# # Get NEON pre-fire reflectance data
# if not os.path.exists(pre_data_path):
#     # Download and save pre-fire data if needed
#     pre_url = (
#         'https://storage.googleapis.com/neon-aop-products/2016/FullSite/D07/'
#         '2016_GRSM_2/L3/Spectrometer/Reflectance/'
#         'NEON_D07_GRSM_DP3_274000_3947000_reflectance.h5')
#     pre_r = requests.get(pre_url)
#     with open(pre_data_path, 'wb') as pre_data_file:
#         pre_data_file.write(pre_r.content)
#         pre_data = pre_r.content
# else:
#     # Read pre-fire data
#     with open(pre_data_path, 'rb') as pre_data_file:
#         pre_data = pre_data_file.read()

In [ ]:
post_fire_refl, post_fire_metadata = refl.aop_h5refl2array(post_data_path)
pre_fire_refl, pre_fire_metadata = refl.aop_h5refl2array(pre_data_path)

In [ ]:
pre_fire_stack = refl.stack_subset_bands(
    pre_fire_refl, pre_fire_metadata, (58, 84, 117, 400))

In [ ]:
post_fire_stack = refl.stack_subset_bands(
    post_fire_refl, post_fire_metadata, (58, 84, 117, 400))

In [ ]:
# Calculate NBR
post_fire_nbr = (
    post_fire_stack[:, :, 2] - post_fire_stack[:, :, 3]) / (
    post_fire_stack[:, :, 2] + post_fire_stack[:, :, 3])

pre_fire_nbr = (
    pre_fire_stack[:, :, 2] - pre_fire_stack[:, :, 3]) / (
    pre_fire_stack[:, :, 2] + pre_fire_stack[:, :, 3])

# Calculate dNBR
dnbr = pre_fire_nbr - post_fire_nbr

In [ ]:
# Plot dNBR
fig, ax = plt.subplots(figsize=(12, 6))

plot = plt.imshow(dnbr,
                  extent=post_fire_metadata['extent'],
                  cmap='PiYG')

cbar = plt.colorbar(plot, aspect=40)
cbar.set_label('Reflectance', rotation=90, labelpad=20)

# Remove scientific notation from tile coordinates
ax.ticklabel_format(useOffset=False, style='plain')

ax.set(title='Chimney Tops 2 Fire dNBR\n (June 2016-October 2017)',
       xlabel='UTM easting (m)',
       ylabel='UTM northing (m)')
rotatexlabels = plt.setp(ax.get_xticklabels(), rotation=90)